In [1]:
# Using a loop and the Pushift PRAW API to gather posts from r/stocks

In [2]:
!pip install praw

In [3]:
import pandas as pd
import requests
import time
import praw

In [4]:
# authentication information:

In [5]:
reddit = praw.Reddit(
    client_id="Jef0auDOdnMZ9vD6zlcWRg",
    client_secret="XqXCuNc590taBMgauwFmMBpDqvZp0A",
    user_agent="poppaearedditapilink",
    redirect_uri="http://localhost:8080",
)
print(reddit.auth.url(["identity"], "...", "permanent"))

https://www.reddit.com/api/v1/authorize?client_id=Jef0auDOdnMZ9vD6zlcWRg&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=identity&state=...


In [6]:
def submissions_pushshift_praw(subreddit, start=None, end=None, limit=10000, extra_query=""):

    # creating an array to contain submissions
    posts = [] 
    


    # preparing api link with {} for inputs specified later
    api = ('https://api.pushshift.io/reddit/submission/search/?subreddit={}&after={}&before={}&sort_type=created_utc&sort=asc&limit={}&q={}')
    api = api.format(subreddit, start, end, limit, extra_query)

    # Using the api to fetch data in JSON form
    data = requests.get(api)
    json_data = data.json()['data']

    # Using a for loop to iterate over each submission, convert them to a praw_submission object, and append them to posts
    for submission in json_data:
        # Take the ID, fetch the PRAW submission object, and append to our list
        praw_submission = reddit.submission(id=submission['id'])
        posts.append(praw_submission)

    return posts


In [7]:
start_date = 1635724800 # 01/09/11 00:00:00
end_date = 1638316800  # 01/10/11 00:00:00

In [8]:
df = pd.DataFrame(columns=['title', 'body', 'created'])

In [ ]:
# checking that the start date is infact before the end date, as a requirement for continuing to run the body of the loop
while start_date < end_date:
    # setting input values for our earlier url
    S = submissions_pushshift_praw(subreddit='stocks', start=start_date, end=end_date)
    # using a for loop over each post:
    for post in S:
        # stopping the loop from fetching deleted or removed posts
        try:
            if post.selftext != '[removed]' and post.selftext != '[deleted]':


                    df = df.append(
                        {'title': post.title,
                         'body': post.selftext,
                         'created': post.created}, ignore_index=True)
        except:
            # telling the loop to continue regardless of errors
            next()

    # telling the loop when to stop
    if len(S) < 100:
        break
    # making sure posts pull from the next earliest date value
    start_date = df['created'].max()
    print(start_date)

In [ ]:
df['created'] = pd.to_datetime(df['created'],unit='s')  # 

In [ ]:
df["text"] = df["title"] + df["body"]

In [ ]:
df = df.drop(["title", "body"], axis = 1)

In [ ]:
!pip install TextBlob
from textblob import TextBlob

# data pre processing from TextBlob
def preprocess(text):
    text = text.str.replace("(<br/>)", "")
    text = text.str.replace('(<a).*(>).*(</a>)', '')
    text = text.str.replace('(&amp)', '')
    text = text.str.replace('(&gt)', '')
    text = text.str.replace('(&lt)', '')
    text = text.str.replace('(\xa0)', ' ')  
    return text
df['text'] = preprocess(df['text'])

df['polarity'] = df['text'].map(lambda text: TextBlob(text).sentiment.polarity)
df['review_len'] = df['text'].astype(str).apply(len)
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

In [ ]:
df = df.drop(["review_len", "word_count", "text"], axis = 1)

In [ ]:
df.to_csv(r'C:\Users\poppa\OneDrive - University of Bristol\3rd\data science\project\r_stocks_data\current\month_polarity\compile\nov3.csv', index = False)

In [ ]:
# Sources
# TextBlob Docs
# Pushift PRAW Docs
# David Little: 4-Step Reddit vs. NY Stock Exchange Sentiment Analysis